In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import json

# Leer archivo de barrios desde ruta relativa directa
barrios = gpd.read_file("googlemaps/data/Barrios/BARRIOS.shp")

# Leer los locales desde el CSV generado previamente
locales = pd.read_csv("data/locales.csv")

# Vista rápida de los datos
barrios.head(1), locales.head(1)

In [ ]:
# Crear mapa centrado en Madrid
m = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Añadir capa de polígonos (barrios)
folium.GeoJson(barrios).add_to(m)

# Añadir capa de locales con cluster de marcadores
marker_cluster = MarkerCluster().add_to(m)

for idx, row in locales.iterrows():
    if pd.notnull(row["latitude"]) and pd.notnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=row.get("name", "Sin nombre"),
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(marker_cluster)

# Mostrar el mapa
m

# Al final de la celda:
m.save("outputs/mapa_madrid_barrios_locales.html")

In [ ]:
import folium
from folium import plugins
import branca.colormap as cm

colormap = cm.LinearColormap(['red', 'orange', 'green'], vmin=1, vmax=5)
colormap.caption = 'Valoración de los negocios'

mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Estilos --> https://leaflet-extras.github.io/leaflet-providers/preview/
    )


# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

# Añadir marcadores como nube de puntos
for _, row in df.iterrows():
    color = colormap(row['valoracion'])
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']} ({row['valoracion']:.1f})"
    ).add_to(mapa)

colormap.add_to(mapa)

mapa

# Al final de la celda:
mapa.save("outputs/mapa_valoracion_puntos.html")

In [ ]:
import folium
from folium.plugins import HeatMap, MiniMap
import branca.colormap as cm

# Crear el mapa base
mapa_densidad = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir el MiniMap
MiniMap().add_to(mapa_densidad)

# Preparar datos para el mapa de calor (sin valoración)
heat_data = [[row['latitud'], row['longitud']] for index, row in df.iterrows()]

# Añadir el mapa de calor
HeatMap(
    heat_data,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_zoom=1
).add_to(mapa_densidad)

# Crear una leyenda simple de densidad
colormap = cm.LinearColormap(
    colors=['lightblue', 'orange', 'red'],
    index=[0, 0.5, 1],
    vmin=0,
    vmax=1
)
colormap.caption = 'Densidad de negocios (baja a alta)'
colormap.add_to(mapa_densidad)

# Mostrar el mapa
mapa_densidad

# Al final de la celda:
mapa_densidad.save("outputs/mapa_densidad.html")

In [ ]:
# Nuevo mapa base para el mapa ponderado
mapa_valorado = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

MiniMap().add_to(mapa_valorado)

# Mapa de calor ponderado por valoración
heat_val = [[row['latitud'], row['longitud'], row['valoracion']] for _, row in df.iterrows()]

HeatMap(
    heat_val,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_zoom=1,
    max_val=5
).add_to(mapa_valorado)

mapa_valorado

# Al final de la celda:
mapa_valorado.save("outputs/mapa_valorado.html")

In [ ]:
import pandas as pd
import folium
import matplotlib.cm as cm
from folium.plugins import MiniMap
from folium import Element

# 1. Cargar el archivo con categorías
df = pd.read_csv(categorizado_csv)

# 2. Eliminar registros sin categoría válida
df = df.dropna(subset=['categoria_negocio'])

# 3. Agrupar por categoría
categoria_info = (
    df.groupby('categoria_negocio')
    .agg(n_negocios=('nombre', 'count'))
    .to_dict(orient='index')
)

# 4. Crear mapa base
mapa_categorias = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir MiniMap
MiniMap().add_to(mapa_categorias)

# 5. Asignar colores únicos por categoría
categorias = sorted(df['categoria_negocio'].unique())
colormap = cm.get_cmap('tab20', len(categorias))

def get_color_categoria(cat):
    idx = categorias.index(cat)
    rgba = colormap(idx)
    rgb = tuple(int(255 * c) for c in rgba[:3])
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# 6. Añadir marcadores
for _, row in df.iterrows():
    color = get_color_categoria(row['categoria_negocio'])
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']}<br>{row['categoria_negocio']}"
    ).add_to(mapa_categorias)

# 7. Leyenda visual por categoría
legend_html = f"""
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 280px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:13px;
    padding: 10px;
    overflow-y: auto;
    max-height: 300px;
">
<b>Tipos de negocio</b><br>
{''.join([
    f"<div style='margin-top:4px;'><span style='background-color:{get_color_categoria(cat)};width:12px;height:12px;display:inline-block;margin-right:6px;'></span>{cat} ({categoria_info[cat]['n_negocios']})</div>"
    for cat in categorias
])}
</div>
"""

mapa_categorias.get_root().html.add_child(Element(legend_html))

# 8. Guardar HTML
mapa_categorias.save("outputs/mapa_categorias.html")
print(f"✅ Mapa por tipo de negocio guardado en: {mapa_categoria_html}")

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MiniMap
import branca.colormap as cm
from shapely.geometry import Point

# Función auxiliar para asegurar CRS coincidente
def ensure_same_crs(gdf1, gdf2):
    """
    Alinea el CRS de gdf2 al de gdf1 si son distintos.
    Devuelve una copia reproyectada de gdf2 si es necesario.
    """
    if gdf1.crs != gdf2.crs:
        print(f"⚠️ CRS diferente detectado: {gdf1.crs} ≠ {gdf2.crs}")
        print("🔄 Reproyectando gdf2 al CRS de gdf1...")
        return gdf2.to_crs(gdf1.crs)
    return gdf2

# 1. Cargar el shapefile de barrios de Madrid (ruta segura desde variable shapefile_path)
gdf_barrios = gpd.read_file(shapefile_path)

# 2. Convertir DataFrame df a GeoDataFrame con puntos
gdf_negocios = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud), crs='EPSG:4326')

# 3. Reproyectar barrios si tienen otro CRS
gdf_barrios = ensure_same_crs(gdf_negocios, gdf_barrios)

# 4. Unión espacial: asignar barrio a cada negocio
gdf_completo = gpd.sjoin(gdf_negocios, gdf_barrios, how='left', predicate='within')

# 5. Contar negocios por barrio
conteo = gdf_completo.groupby('NOMBRE').size().reset_index(name='num_negocios')
gdf_barrios = gdf_barrios.merge(conteo, on='NOMBRE', how='left')
gdf_barrios['num_negocios'] = gdf_barrios['num_negocios'].fillna(0)

# 6. Normalizar densidad
gdf_barrios['densidad_norm'] = (
    gdf_barrios['num_negocios'] - gdf_barrios['num_negocios'].min()
) / (
    gdf_barrios['num_negocios'].max() - gdf_barrios['num_negocios'].min()
)

# 7. Crear el mapa base
mapa_barrios = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=12,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# 8. Añadir MiniMap
MiniMap().add_to(mapa_barrios)

# 9. Crear leyenda
colormap = cm.LinearColormap(['lightblue', 'orange', 'red'], vmin=0, vmax=1)
colormap.caption = 'Densidad normalizada de negocios por barrio'
colormap.add_to(mapa_barrios)

# 10. Dibujar los polígonos
folium.GeoJson(
    gdf_barrios,
    name="Barrios por densidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['densidad_norm']),
        'color': 'black',
        'weight': 0.7,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMBRE", "num_negocios"],
        aliases=["Barrio", "Nº negocios"],
        localize=True
    )
).add_to(mapa_barrios)

# 11. Guardar el mapa de barrios en HTML
mapa_barrios.save("outputs/mapa_barrios.html")

# 12. Mostrar el mapa interactivo
mapa_barrios


In [ ]:
import folium
import joblib

# 1) Cargar el modelo entrenado
clf = joblib.load('models/rf_classifier.pkl')

# 2) Definir columnas de features (incluyendo las nuevas)
feature_cols = [
    'latitud','longitud','categoria_negocio',
    'dist_city_center_km','local_density_1km','cluster_zone'
]

# 3) Predecir clase y probabilidad máxima
df['pred_clase'] = clf.predict(df[feature_cols])
df['pred_proba'] = clf.predict_proba(df[feature_cols]).max(axis=1)

# 4) Crear mapa base centrado en city_center
m_pred = folium.Map(location=city_center, zoom_start=12)

# 5) Añadir marcadores coloreados por predicción
colors = {'baja':'red','media':'orange','alta':'green'}
for _, r in df.iterrows():
    folium.CircleMarker(
        location=(r.latitud, r.longitud),
        radius=4,
        color=colors.get(r.pred_clase, 'blue'),
        fill=True,
        fill_color=colors.get(r.pred_clase, 'blue'),
        fill_opacity=0.7,
        popup=(
            f"ID: {r.get('id_local','-')}<br>"
            f"Predicción: {r.pred_clase} ({r.pred_proba:.2f})<br>"
            f"Categoría: {r.categoria_negocio}"
        )
    ).add_to(m_pred)

# 6) Mostrar mapa con predicciones
m_pred

# Al final de la celda:
m_pred.save("outputs/mapa_predicciones_modelo.html")

### MAPAS CON DATOS UNICOS DE RESTAURACION

## 🌍 Visualización Interactiva de Negocios

Con ayuda de **Folium**, se genera un mapa interactivo con los siguientes elementos:

- Marcadores con información detallada del negocio.
- Un **MiniMapa** que facilita la navegación.
- Estilo de mapa personalizado usando `CartoDB.VoyagerLabels`.

Cada negocio se representa con un icono personalizado, y al hacer clic se despliega un resumen con nombre, dirección, puntuación y tipo.

from pathlib import Path
import folium
from folium import plugins
import pandas as pd

# Ruta al CSV de locales de restauración
locales_restauracion_csv = Path("../data/locales_restauracion.csv")  # Ajusta si es necesario
df = pd.read_csv(locales_restauracion_csv)
df = df.dropna(subset=['latitud', 'longitud'])

# Crear el mapa base centrado en los locales
mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

# Añadir marcadores individuales con popup informativo
for _, row in df.iterrows():
    popup_text = f"""
    <b>{row.get('nombre', 'Sin nombre')}</b><br>
    Dirección: {row.get('direccion', 'Sin dirección')}<br>
    Rating: {row.get('rating', 'N/A')} ({row.get('user_ratings_total', 0)} reseñas)<br>
    Tipo: {row.get('types', 'No especificado')}
    """
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        icon=plugins.BeautifyIcon(
            icon="cutlery",
            icon_shape="circle",
            border_color='green',
            text_color="green",
            background_color='white'
        ),
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['nombre']
    ).add_to(mapa)

# Guardar el mapa con nombre específico para evitar sobrescribir el general
mapa_locales_restauracion_html = Path("../outputs/mapa_locales_restauracion.html")
mapa.save(mapa_locales_restauracion_html)

# Mostrar en el notebook
mapa

## 🗺️ Mapa Interactivo de Negocios con Valoración

Se genera un mapa interactivo con `folium` para visualizar geográficamente la distribución de los negocios, codificados por color según su **valoración ajustada**.

### Características del mapa:

- 🟢 **Color**: El color de cada punto indica la valoración (escala de rojo a verde).
- 🔍 **MiniMap**: Se incluye una vista general secundaria en la esquina inferior derecha para facilitar la navegación.
- 📍 **Marcadores**: Cada negocio aparece como un punto circular con información emergente (`popup`) mostrando su nombre y valoración.
- 🎨 **Leyenda de color**: Se añade un colormap con la escala de valores (1 a 5) como referencia visual.

> 🧭 Esta visualización permite identificar zonas con mayor o menor concentración de negocios bien valorados y puede ayudar en análisis espaciales o decisiones estratégicas.

In [ ]:
import folium
from folium import plugins
import branca.colormap as cm
import pandas as pd

# Crear colormap para la valoración (rating)
colormap = cm.LinearColormap(['red', 'orange', 'green'], vmin=1, vmax=5)
colormap.caption = 'Valoración media de los negocios'

# Crear el mapa centrado en el promedio de latitud y longitud
mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Otros estilos: https://leaflet-extras.github.io/leaflet-providers/preview/
)

# Añadir un minimapa
plugins.MiniMap().add_to(mapa)

# Añadir marcadores como nube de puntos coloreados por rating
for _, row in df.iterrows():
    rating = row.get('rating', None)
    if pd.isna(rating):
        continue  # Omitir si no hay valoración

    color = colormap(rating)
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']} ({rating:.1f})"
    ).add_to(mapa)

# Añadir leyenda al mapa
colormap.add_to(mapa)

# Mostrar mapa
mapa

# Al final de la celda:
mapa.save("outputs/mapa_valoracion_colormap.html")

## 🧭 Mapa de Densidad de Negocios por Barrio (Shapefile)

Se genera un mapa coroplético basado en un shapefile de barrios de Madrid, que representa la **densidad relativa de negocios por zona**. Este análisis espacial permite detectar patrones geográficos y desequilibrios en la distribución comercial.

### Fases del proceso:

1. 🗂️ **Carga del shapefile** con geometrías de los barrios (`shapefile_path`).
2. 📍 **Conversión de coordenadas** del dataframe `df` a un `GeoDataFrame`.
3. 🔄 **Unificación de sistemas de referencia (CRS)** entre capas geoespaciales.
4. 🧩 **Unión espacial (spatial join)** para asignar cada negocio a su barrio.
5. 📊 **Conteo de negocios por barrio** y fusión con el shapefile.
6. 📈 **Normalización** de la cantidad de negociÇos para representar una escala relativa de densidad.
7. 🗺️ **Creación del mapa base** con `folium` y estilos personalizados.
8. 🌈 **Leyenda de colores** que representa la densidad relativa (de azul claro a rojo).
9. 🧱 **Dibujo de polígonos** con `GeoJson`, mostrando nombre del barrio y número de negocios.
10. 💾 **Exportación del mapa** como archivo HTML interactivo.

> 📌 Este tipo de visualización permite realizar análisis territoriales precisos y detectar barrios con mayor o menor concentración de actividad comercial.

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MiniMap
import branca.colormap as cm
from shapely.geometry import Point

# --- Función auxiliar para asegurar que los CRS coinciden ---
def ensure_same_crs(gdf1, gdf2):
    """
    Alinea el CRS de gdf2 al de gdf1 si son distintos.
    Devuelve una copia reproyectada de gdf2 si es necesario.
    """
    if gdf1.crs != gdf2.crs:
        print(f"⚠️ CRS diferente detectado: {gdf1.crs} ≠ {gdf2.crs}")
        print("🔄 Reproyectando gdf2 al CRS de gdf1...")
        return gdf2.to_crs(gdf1.crs)
    return gdf2

# --- 1. Cargar el shapefile de barrios de Madrid ---
gdf_barrios = gpd.read_file(shapefile_path)

# --- 2. Cargar locales desde el CSV de restauración ---
df = pd.read_csv(locales_restauracion_csv)
df = df.dropna(subset=['latitud', 'longitud'])

# --- 3. Convertir df a GeoDataFrame ---
gdf_negocios = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud), crs='EPSG:4326')

# --- 4. Reproyectar barrios si es necesario ---
gdf_barrios = ensure_same_crs(gdf_negocios, gdf_barrios)

# --- 5. Unión espacial: asignar barrio a cada negocio ---
gdf_completo = gpd.sjoin(gdf_negocios, gdf_barrios, how='left', predicate='within')

# --- 6. Contar negocios por barrio ---
conteo = gdf_completo.groupby('NOMBRE').size().reset_index(name='num_negocios')
gdf_barrios = gdf_barrios.merge(conteo, on='NOMBRE', how='left')
gdf_barrios['num_negocios'] = gdf_barrios['num_negocios'].fillna(0)

# --- 7. Normalizar densidad de negocios por barrio ---
gdf_barrios['densidad_norm'] = (
    gdf_barrios['num_negocios'] - gdf_barrios['num_negocios'].min()
) / (
    gdf_barrios['num_negocios'].max() - gdf_barrios['num_negocios'].min()
)

# --- 8. Crear el mapa base ---
mapa_barrios = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=12,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# --- 9. Añadir MiniMap ---
MiniMap().add_to(mapa_barrios)

# --- 10. Crear leyenda de colores personalizada ---
colormap = cm.LinearColormap(['lightblue', 'orange', 'red'], vmin=0, vmax=1)
colormap.caption = 'Densidad normalizada de locales de restauración por barrio'
colormap.add_to(mapa_barrios)

# --- 11. Dibujar polígonos con estilo y tooltip actualizado ---
folium.GeoJson(
    gdf_barrios,
    name="Barrios por densidad de restauración",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['densidad_norm']),
        'color': 'black',
        'weight': 0.7,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMBRE", "num_negocios"],
        aliases=["Barrio", "Nº locales de restauración"],
        localize=True
    )
).add_to(mapa_barrios)

# --- 12. Guardar mapa HTML con nombre específico para restauración ---
mapa_barrios.save("outputs/mapa_barrios_restauracion.html")

# --- 13. Mostrar mapa en notebook ---
mapa_barrios